# Gradient Boosted Tree Model with XGBoost

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score, classification_report
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
from xgboost import to_graphviz
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import numpy as np
import graphviz
import joblib

In [8]:
X_train = pd.read_csv("../../Data/preprocessed_X_train.csv", index_col="respondent_id")
X_test = pd.read_csv("../../Data/preprocessed_X_test.csv", index_col="respondent_id")
y_train = pd.read_csv("../../Data/preprocessed_y_train.csv", index_col="respondent_id")
y_test = pd.read_csv("../../Data/preprocessed_y_test.csv", index_col="respondent_id")

In [9]:
# Espacio de búsqueda de hiperparámetros
search_space = {
    'estimator__max_depth': Integer(3, 20),  # Aumentar profundidad máxima del árbol
    'estimator__learning_rate': Real(0.001, 0.3, prior='log-uniform'),  # Ampliar rango del learning rate
    'estimator__subsample': Real(0.3, 1.0),  # Permitir tamaños de muestra más pequeños
    'estimator__colsample_bytree': Real(0.3, 1.0),  # Más flexibilidad en columnas seleccionadas
    'estimator__n_estimators': Integer(50, 500),  # Incrementar número máximo de árboles
    'estimator__gamma': Real(0.0, 10.0),  # Permitir mayor regularización
    'estimator__min_child_weight': Integer(1, 10),  # Considerar pesos mínimos para evitar sobreajuste
    'estimator__reg_alpha': Real(0.0, 1.0),  # Regularización L1
    'estimator__reg_lambda': Real(0.0, 1.0)  # Regularización L2
}

# Crear el modelo base
xgb_model = XGBClassifier(eval_metric='auc', random_state=42)

# Envolverlo en MultiOutputClassifier para predicción multietiqueta
multi_xgb = MultiOutputClassifier(xgb_model)

In [10]:
# Configurar BayesSearchCV
opt = BayesSearchCV(
    multi_xgb,
    search_space,
    cv=5,
    n_iter=50,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1
)

# Entrenar el modelo con optimización
opt.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", opt.best_params_)
print("Mejor puntaje (AUROC):", opt.best_score_)

# Guardar el mejor modelo optimizado
joblib.dump(opt.best_estimator_, "Modelos/XGBoost_best_model.pkl")
print("Modelo guardado como 'XGBoost_best_model.pkl'")

Mejores hiperparámetros: OrderedDict({'estimator__colsample_bytree': 0.9923319362435739, 'estimator__gamma': 0.20432997478467166, 'estimator__learning_rate': 0.018830734677431557, 'estimator__max_depth': 5, 'estimator__min_child_weight': 5, 'estimator__n_estimators': 494, 'estimator__reg_alpha': 0.3347004937222214, 'estimator__reg_lambda': 0.9746608636023493, 'estimator__subsample': 0.37139687975992147})
Mejor puntaje (AUROC): 0.8658933293740831
Modelo guardado como 'XGBoost_best_model.pkl'


In [ ]:
# Obtener la importancia de las características del modelo base
model_base = opt.best_estimator_.estimators_[0] 
feature_importances = model_base.feature_importances_

# Crear un DataFrame para visualizar la importancia
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

# Mostrar las características más importantes
print(importance_df)

# Establecer un umbral (por ejemplo, 0.001) para filtrar características poco importantes
threshold = 0.001
important_features = importance_df[importance_df['importance'] > threshold]['feature'].tolist()

print(f"Características seleccionadas ({len(important_features)}):", important_features)

# Filtrar el conjunto de datos con las características seleccionadas
X_train_filtered = X_train[important_features]
X_test_filtered = X_test[important_features]

                           feature  importance
84                doctor_recc_h1n1    0.116306
112    health_insurance_is_missing    0.039825
89                health_insurance    0.031491
90     opinion_h1n1_vacc_effective    0.028797
91               opinion_h1n1_risk    0.026262
..                             ...         ...
39    employment_industry_qnlwzans    0.000000
37    employment_industry_phxvnwax    0.000000
35    employment_industry_msuufmds    0.000000
29    employment_industry_dotnnunm    0.000000
120  household_children_is_missing    0.000000

[121 rows x 2 columns]
Características seleccionadas (95): ['doctor_recc_h1n1', 'health_insurance_is_missing', 'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'employment_industry_haxffmxo', 'health_worker', 'opinion_h1n1_sick_from_vacc_is_missing', 'employment_industry_fcxhlnwr', 'opinion_seas_risk', 'employment_occupation_cmhcxjea', 'employment_occupation_haliazsg', 'doctor_recc_seasonal', 'marital_status_

In [12]:
# Crear el modelo base
xgb_model2 = XGBClassifier(eval_metric='auc', random_state=42)

# Envolverlo en MultiOutputClassifier para predicción multietiqueta
multi_xgb2 = MultiOutputClassifier(xgb_model2)

# Configurar BayesSearchCV
opt2 = BayesSearchCV(
    multi_xgb,
    search_space,
    cv=5,
    n_iter=50,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1
)

# Entrenar el modelo con optimización
opt2.fit(X_train_filtered, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", opt2.best_params_)
print("Mejor puntaje (AUROC):", opt2.best_score_)

# Guardar el mejor modelo optimizado
# joblib.dump(opt2.best_estimator_, "Modelos/XGBoost_best_model.pkl")
# print("Modelo guardado como 'XGBoost_best_model.pkl'")

KeyboardInterrupt: 

In [13]:
# Predicción
y_pred = opt.best_estimator_.predict(X_test)

# Calcular probabilidades para AUROC
y_proba = opt.best_estimator_.predict_proba(X_test)

# Convertir las probabilidades a un DataFrame para cada etiqueta
y_proba_h1n1 = y_proba[0][:, 1]  # Para H1N1
y_proba_seasonal = y_proba[1][:, 1]  # Para vacuna estacional

# Extraer las etiquetas reales como arrays 1D
y_test_h1n1 = y_test.iloc[:, 0].values  # H1N1
y_test_seasonal = y_test.iloc[:, 1].values  # Vacuna estacional

# AUROC para cada etiqueta
roc_auc_h1n1 = roc_auc_score(y_test_h1n1, y_proba_h1n1)
roc_auc_seasonal = roc_auc_score(y_test_seasonal, y_proba_seasonal)

print(f"AUROC para H1N1: {roc_auc_h1n1}")
print(f"AUROC para vacuna estacional: {roc_auc_seasonal}")

# Informe de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

AUROC para H1N1: 0.8777441415555572
AUROC para vacuna estacional: 0.8646201134753051
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.74      0.53      0.61      1135
           1       0.79      0.76      0.77      2488

   micro avg       0.77      0.69      0.73      3623
   macro avg       0.76      0.64      0.69      3623
weighted avg       0.77      0.69      0.72      3623
 samples avg       0.35      0.34      0.34      3623



c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VEN

In [15]:
# Cargar el dataset de prueba preprocesado
test_set_preprocessed = pd.read_csv("../../Data/preprocessed_test_set_features.csv")

# Asegurarse de que respondent_id esté disponible
respondent_ids = test_set_preprocessed['respondent_id']

# Eliminar respondent_id del conjunto de características
X_test_final = test_set_preprocessed.drop(columns=['respondent_id'])

# Realizar predicciones de probabilidad
y_proba_test = opt.best_estimator_.predict_proba(X_test_final)

# Extraer las probabilidades para la clase positiva (1)
y_proba_h1n1_test = y_proba_test[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal_test = y_proba_test[1][:, 1]  # Probabilidades para vacuna estacional

# Crear el DataFrame de submission
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

# Guardar el archivo de submission
submission.to_csv("Submissions/XGBoost_submission.csv", index=False)
print("Archivo 'XGBoost_submission.csv' generado correctamente.")

Archivo 'XGBoost_submission.csv' generado correctamente.


Ejemplo de carga de modelo guardado para su posterior uso.

In [ ]:
# Cargar el modelo guardado
loaded_model = joblib.load("Modelos/XGBoost_best_model.pkl")
print("Modelo cargado correctamente.")

# Realizar predicciones usando el modelo cargado
y_proba_test = loaded_model.predict_proba(X_test_final)

# Extraer probabilidades para cada etiqueta
y_proba_h1n1_test = y_proba_test[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal_test = y_proba_test[1][:, 1]  # Probabilidades para vacuna estacional

# Crear un DataFrame para la submission si es necesario
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

Modelo cargado correctamente.
